# Spectral Fit

In [ ]:
import sherpa.astro.ui as sau
import matplotlib.pyplot as plt
from sherpa_contrib.chart import save_chart_spectrum

In [ ]:
sau.reset()

In [ ]:
sau.show_all()

In [ ]:
DATASET_IDS = [f"obs-id-{index.obs_id}" for index in context.file_indices]

In [ ]:
for index in context.file_indices:
    dataset_id = f"obs-id-{index.obs_id}"

    sau.load_data(dataset_id, str(index.paths_spectra_pha[irf_label] / f"{irf_label}.pi"))
    sau.group_counts(dataset_id, 10)

sau.notice(0.5, 7)

In [ ]:
sau.set_stat("wstat")
sau.set_method("simplex")

In [ ]:
for dataset_id in DATASET_IDS:
    sau.set_source(dataset_id, sau.xsphabs.absorption * sau.powlaw1d.pwl)

sau.xsphabs.absorption.nh.val = 0.09
sau.xsphabs.absorption.nh.frozen = True
sau.powlaw1d.pwl.ampl.val = 0.001
sau.powlaw1d.pwl.gamma.val = 1.5

In [ ]:
sau.fit()

In [ ]:
for dataset_id in DATASET_IDS:
    sau.plot_fit_resid(dataset_id)
    plt.show()

In [ ]:
for dataset_id in DATASET_IDS:
    sau.set_pileup_model(dataset_id, sau.jdpileup.jdp)

sau.jdpileup.jdp.f.min = 0.85
sau.jdpileup.jdp.ftime = 0.6
sau.jdpileup.jdp.fracexp = 0.987

In [ ]:
sau.fit()

In [ ]:
for dataset_id in DATASET_IDS:
    sau.plot_fit_resid(dataset_id)
    plt.show()

In [ ]:
for dataset_id in DATASET_IDS:
    sau.set_analysis(dataset_id, "energy", "rate", factor=1)

e_min = 0.5
e_max = 7

for index in context.file_indices:
    filename = index.filenames_spectra[irf_label]
    save_chart_spectrum(
        str(filename), elow=e_min, ehigh=e_max, clobber=True, id=DATASET_IDS[0]
    )